In [1]:
import pandas as pd
import datetime
import datetime
import os
from haversine import haversine
import glob
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

IRI_Selected_18=['LOS ANGELES','DALLAS/FT WORTH','NEW YORK','HOUSTON','WEST TEX/NEW MEXICO','PHOENIX/TUCSON','SOUTH CAROLINA',
                'BIRMINGHAM/MONTGOMERY','MISSISSIPPI','BALTIMORE/WASHINGTON','NEW ORLEANS/MOBILE',
                 'CHICAGO','ATLANTA','DETROIT','MIAMI/FT LAUDERDALE','HARRISBURG/SCRANTON','BOSTON',"ROANOKE"] #'RALEIGH/GREENSBORO','DENVER'
len(IRI_Selected_18)

18

# IRI zips for the 18 Mkts

In [2]:
IRI_Counties_both=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Sheet1',dtype=str)
IRI_Counties_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='IRI',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Nielsen',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen.columns=[['County','cleaned_name']]
IRI_Counties_both=IRI_Counties_both[(IRI_Counties_both['IRI_Label']!='nan') & (IRI_Counties_both['Nielsen_Label']!='nan')]

IRI_Counties_both=IRI_Counties_both[['County','County']]
IRI_Counties_both.columns=['County','cleaned_name']

Nielsen_counties=IRI_Counties_both.append(IRI_Counties_Nielsen).drop_duplicates().reset_index()
IRI_counties=IRI_Counties_both.append(IRI_Counties_IRI).drop_duplicates().reset_index()

del Nielsen_counties['index']
del IRI_counties['index']


In [3]:
All_DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
All_DMA_zips=All_DMA_zips.iloc[:,[0,2,6,9]]
All_DMA_zips.columns=['zip_code','DMA','State','County']
All_DMA_zips['County']=All_DMA_zips['County']+"|"+All_DMA_zips['State']
All_DMA_zips['County']=All_DMA_zips['County'].apply(lambda x: x.replace(" ","").lower())
All_DMA_zips=All_DMA_zips[~All_DMA_zips['State'].isin(['AK','HI'])]

In [4]:
All_IRI_counties=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",dtype=str,sheetname='U.S. Counties',skiprows=1)
All_IRI_counties=All_IRI_counties[['County Name','State','IRI Market']]
All_IRI_counties['County']=All_IRI_counties['County Name']+"|"+All_IRI_counties['State']
All_IRI_counties['County']=All_IRI_counties['County'].apply(lambda x: x.replace(" ","").lower())
All_IRI_counties=pd.merge(All_IRI_counties,IRI_counties,on="County",how="left")
All_IRI_counties=All_IRI_counties[['IRI Market','cleaned_name']]

In [5]:
All_DMA_zips=pd.merge(All_DMA_zips,Nielsen_counties,on="County",how="left")
All_DMA_zips=All_DMA_zips[['zip_code','DMA','County','cleaned_name']].rename(columns={"County":"Nielsen_County"})
All_DMA_zips_merge=pd.merge(All_DMA_zips,All_IRI_counties,on="cleaned_name",how="left")


In [6]:
IRI_Zip_with_dup=All_DMA_zips_merge[['IRI Market','zip_code']].drop_duplicates()

In [7]:
IRI_Zip_18=IRI_Zip_with_dup[IRI_Zip_with_dup['IRI Market'].isin(IRI_Selected_18)]
print(IRI_Zip_18.shape)
print(IRI_Zip_18['IRI Market'].nunique())
print(IRI_Zip_18['zip_code'].nunique())


(10157, 2)
18
10149


# Walmarts in the zips above from SafeGraph

In [8]:
latest_SG_brand_info=pd.read_csv("/home/jian/Docs/SafeGraph/places_20190629/brand_info/part-00000-tid-3844431843075532209-d890661b-f976-44e8-9678-725e8cd08c6f-119-c000.csv.gz",
                                dtype=str,compression="gzip",sep="|")
[x for x in latest_SG_brand_info['brand_name'].unique().tolist() if "almart" in x.lower()]

['Walmart', 'Walmart inc']

In [9]:
POI_data=glob.glob("/home/jian/Docs/SafeGraph/places_20190629/poi/*csv.gz")
POI_data

['/home/jian/Docs/SafeGraph/places_20190629/poi/part-00001-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-112-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00003-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-114-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00000-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-111-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00004-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-115-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00002-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-113-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00005-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-116-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00007-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-118-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places

In [10]:
all_SG_WMT_POI=pd.DataFrame()
all_SG_WMTinc_POI=pd.DataFrame()

for file in POI_data:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep="|")
    df_Walmart=df[df['brands']=="Walmart"]
    df_Walmartinc=df[df['brands']=="Walmart inc"]
    
    all_SG_WMT_POI=all_SG_WMT_POI.append(df_Walmart)
    all_SG_WMTinc_POI=all_SG_WMTinc_POI.append(df_Walmartinc)
    
    print(datetime.datetime.now(),file)

2019-09-09 17:51:33.143987 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00001-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-112-c000.csv.gz
2019-09-09 17:51:59.241662 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00003-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-114-c000.csv.gz
2019-09-09 17:52:11.035351 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00000-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-111-c000.csv.gz
2019-09-09 17:52:21.919095 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00004-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-115-c000.csv.gz
2019-09-09 17:52:32.445962 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00002-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-113-c000.csv.gz
2019-09-09 17:52:42.540935 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00005-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-116-c000.csv.gz
2019-09-09 17:52:52.244493 /home/j

In [11]:
print(all_SG_WMT_POI.shape)
print(all_SG_WMTinc_POI.shape)

# all_SG_WMT_POI

(4669, 20)
(0, 20)


In [12]:
all_SG_WMT_POI['brands'].unique()

array(['Walmart'], dtype=object)

In [13]:
df_slicing_Wmts=all_SG_WMT_POI.groupby("location_name")['safegraph_place_id'].count().to_frame().reset_index().sort_values("safegraph_place_id")
included_location_names_list=df_slicing_Wmts[df_slicing_Wmts['safegraph_place_id']>=10]['location_name'].tolist()
print(included_location_names_list)

all_SG_WMT_POI=all_SG_WMT_POI[all_SG_WMT_POI["location_name"].isin(included_location_names_list)]
all_SG_WMT_POI['zip_code']=all_SG_WMT_POI['zip_code'].apply(lambda x: x.zfill(5))
all_SG_WMT_POI.shape

['Walmart Supercenter', 'Wal Mart Supercenter', 'Walmart']


(4653, 20)

In [14]:
df_zip_by_18_mkt_count=IRI_Zip_18.groupby("zip_code")['IRI Market'].nunique().to_frame().reset_index().rename(columns={"IRI Market":"count_mkts"}).sort_values("count_mkts",ascending=False)
df_zip_by_18_mkt_list=IRI_Zip_18.groupby("zip_code")['IRI Market'].apply(list).to_frame().reset_index()

df_zip_by_18_mkts=pd.merge(df_zip_by_18_mkt_count,df_zip_by_18_mkt_list,on="zip_code",how="outer")
df_zip_by_18_mkts.shape

(10149, 3)

In [15]:
walmarts_with_IRIs=pd.merge(all_SG_WMT_POI,df_zip_by_18_mkts,on="zip_code",how="left")
walmarts_with_IRIs['IRI Market']=walmarts_with_IRIs['IRI Market'].fillna("['Others_out_18']")
walmarts_with_IRIs['IRI Market']=walmarts_with_IRIs['IRI Market'].astype(str)
walmarts_with_IRIs['IRI Market']=walmarts_with_IRIs['IRI Market'].apply(eval)
walmarts_with_IRIs['latitude']=walmarts_with_IRIs['latitude'].astype(float)
walmarts_with_IRIs['longitude']=walmarts_with_IRIs['longitude'].astype(float)

walmarts_with_IRIs.head(2)

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,...,state,zip_code,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,count_mkts,IRI Market
0,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,ca,95776,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,NaN,[Others_out_18]
1,sg:a34b1824bf7c4d758634a35173905c85,NaN,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,28.055297,-80.623241,...,fl,32901,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...","POLYGON ((-80.6234859 28.055599800000003, -80....",OWNED_POLYGON,NaN,false,NaN,NaN,[Others_out_18]


In [16]:
walmarts_with_IRIs.shape

(4653, 22)

In [17]:
walmarts_with_IRIs['safegraph_place_id'].nunique()

4653

In [18]:
walmarts_with_IRIs_iter=walmarts_with_IRIs[['safegraph_place_id','latitude','longitude']]

In [19]:
df_mapping_store_10_zips=pd.DataFrame()
for ind,row in walmarts_with_IRIs_iter.iterrows():
    store_coor=[row['latitude'],row['longitude']]
    store_id=row['safegraph_place_id']
    for zip_cd in zip_centers.keys():
        dist=haversine(store_coor,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"zip_cd":zip_cd,"dist_miles":dist},index=[store_id])
            df_mapping_store_10_zips=df_mapping_store_10_zips.append(df)

In [20]:
df_mapping_store_10_zips=df_mapping_store_10_zips.reset_index().rename(columns={"index":"safegraph_place_id"})

In [21]:
df_output=pd.merge(walmarts_with_IRIs,df_mapping_store_10_zips,on="safegraph_place_id",how="left")
df_output=df_output.rename(columns={"zip_code":"store_zip"})

df_output.head(2)

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,...,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,count_mkts,IRI Market,dist_miles,zip_cd
0,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,NaN,[Others_out_18],7.889054,95618
1,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,NaN,[Others_out_18],1.005072,95776


In [22]:
df_output=df_output[['IRI Market','safegraph_place_id','zip_cd','dist_miles']+[x for x in df_output.columns.tolist() if x not in ['IRI Market','safegraph_place_id','zip_cd','dist_miles'] ]]
df_output['iri_list_str']=df_output['IRI Market'].astype(str)
df_output=df_output.sort_values(['iri_list_str','safegraph_place_id','dist_miles'])
df_output.shape

(96993, 25)

In [23]:
df_output_unique_zip_by_IRI=df_output[['iri_list_str','zip_cd']].drop_duplicates()
df_output_unique_zip_by_IRI=df_output_unique_zip_by_IRI[pd.notnull(df_output_unique_zip_by_IRI['zip_cd'])]
df_output_unique_zip_by_IRI['iri_list_str']=df_output_unique_zip_by_IRI['iri_list_str'].apply(eval)
df_output_unique_zip_by_IRI_agg=pd.DataFrame()
for ind,row in df_output_unique_zip_by_IRI.iterrows():
    df=pd.DataFrame({"IRI_mkt_of_walmart":row['iri_list_str'],"zip_cd":[row['zip_cd']]*len(row['iri_list_str'])},index=[x for x in range(len(row['iri_list_str']))])
    df_output_unique_zip_by_IRI_agg=df_output_unique_zip_by_IRI_agg.append(df)
    
df_output_unique_zip_by_IRI_agg.shape

(24490, 2)

In [24]:
df_output_unique_zip_by_IRI_agg=df_output_unique_zip_by_IRI_agg.sort_values("IRI_mkt_of_walmart").drop_duplicates(["IRI_mkt_of_walmart","zip_cd"])

In [25]:
df_national_zips_18=df_output_unique_zip_by_IRI_agg[df_output_unique_zip_by_IRI_agg['IRI_mkt_of_walmart']!="Others_out_18"]
df_national_zips_18['IRI_mkt_of_walmart']="Selected_18_Mkts"
df_national_zips_18=df_national_zips_18.drop_duplicates()
df_national_zips_others=df_output_unique_zip_by_IRI_agg[df_output_unique_zip_by_IRI_agg['IRI_mkt_of_walmart']=="Others_out_18"]
df_national_zips_others=df_national_zips_others[~df_national_zips_others['zip_cd'].isin(df_national_zips_18['zip_cd'].tolist())]
df_national_zips_others=df_national_zips_others.drop_duplicates()
df_output_national=df_national_zips_18.append(df_national_zips_others)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
print(datetime.datetime.now())
df_output_unique_zip_by_IRI.shape

2019-09-09 18:04:33.953923


(24481, 2)

In [28]:
df_output_unique_zip_by_IRI['zip_cd'].apply(lambda x: len(x)).unique()

array([5])

# Add demo

In [29]:
excel_demo=pd.ExcelFile("./SunnyD_target_demo_by_zip_JL_2019-09-05.xlsx")
excel_demo.sheet_names

['based_on_EASI_zip4', 'based_on_census', 'both_sidebyside']

In [30]:
# EASI
df_easi_demo=excel_demo.parse("based_on_EASI_zip4")
df_easi_demo['ZIP_CODE']=df_easi_demo['ZIP_CODE'].apply(lambda x: str(x).zfill(5))
df_easi_demo=df_easi_demo.rename(columns={"ZIP_CODE":"zip_cd"})

In [31]:
df_easi_demo.head(2)

,zip_cd,teenager_population,mother_population,HH_inc_avg
0,00501,NaN,NaN,NaN
1,01001,312.433168,893.364506,93533.973313


In [32]:
df_output_national_EASI=pd.merge(df_output_national,df_easi_demo,on="zip_cd",how="left")
df_output_unique_zip_by_IRI_agg_EASI=pd.merge(df_output_unique_zip_by_IRI_agg,df_easi_demo,on="zip_cd",how="left")

In [33]:
# Census
df_census_demo=excel_demo.parse("based_on_census")
df_census_demo['ZIP_CODE']=df_census_demo['ZIP_CODE'].apply(lambda x: str(x).zfill(5))
df_census_demo=df_census_demo[['ZIP_CODE','teenagers_10_19_census','calculated_Monther_census','HH_inc_avg']]
df_census_demo=df_census_demo.rename(columns={"ZIP_CODE":"zip_cd"})

In [34]:
df_output_national_census=pd.merge(df_output_national,df_census_demo,on="zip_cd",how="left")
df_output_unique_zip_by_IRI_agg_census=pd.merge(df_output_unique_zip_by_IRI_agg,df_census_demo,on="zip_cd",how="left")

In [35]:
writer=pd.ExcelWriter("./df_zips_10_miles_around_all_walmarts_labled_IRI_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_national_EASI.to_excel(writer,"unique_zips_10miles_wmt_EASI",index=False)
df_output_unique_zip_by_IRI_agg_EASI.to_excel(writer,"unique_zips_by_wmt_IRI_EASI",index=False)

df_output_national_census.to_excel(writer,"unique_zips_10miles_wmt_Census",index=False)
df_output_unique_zip_by_IRI_agg_census.to_excel(writer,"unique_zips_by_wmt_IRI_Census",index=False)

df_output.to_excel(writer,"all_zips_with_store_info_POI",index=False)

writer.save()

In [36]:
df_output_unique_zip_by_IRI_agg_census['zip_cd'].apply(lambda x: len(x)).unique()

array([5])